In [2]:
import os
import json
import cv2
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from utils.court import load_court_mask, load_court_poi
from utils.transform import map_court_to_frame
from dataset_utils.preparation import generate_requests, calculate_homography, generate_onehot, \
calculate_reprojection_rmse, find_nonzero_points, rescale_theta, NumpyEncoder, FOOTBALL_PITCH_IGNORE_POINTS

**Drawing function for debugging**

In [3]:
def draw_poi(img, poi):
    H, W = img.shape[0:2]
    scaled_poi = np.copy(poi)
    scaled_poi[:,0] *= W
    scaled_poi[:,1] *= H
    
    for pt in scaled_poi:
        x, y = int(round(pt[0])), int(round(pt[1]))
        img = cv2.circle(img, (x, y), 5, color=(255, 0, 255), thickness=-1)
    
    return img


def overlay(img1, img2, alpha=0.3):
    m = cv2.inRange(img2, (0, 0, 0), (0, 0, 0))
    m = cv2.merge([m, m, m])
    overlaid = (img1 & m) + img2 * alpha + (img1 & (255 - m)) * (1 - alpha)

    return overlaid.astype('uint8')


def draw_text(img, text, pos, color=(255, 255, 255), scale=0.75, lineType=1,
              font=cv2.FONT_HERSHEY_COMPLEX_SMALL):
    cv2.putText(img, text, pos, font, scale, color, lineType)

    
def debug_draw(img, mask, poi=None, rmse=None):
    canvas = overlay(img, mask)
    
    if poi is not None:
        canvas = draw_poi(canvas, poi)
        
    if rmse is not None:
        draw_text(canvas, '{:.5f}'.format(rmse), (20, 20), color=(0,0,255), scale=1)
        
    return canvas

**The following steps are required to create a dataset for training a field homography model:**
1. Generate requests (*game, frame id* and *manual PoI\**) from annotations for further processing 
2. Calculate homography (*theta*) from manually annotated *PoI* of a field
3. Project the field *PoI* to the frame *PoI* using the homography
4. Calculate the *Reprojection RMSE*** based on the reprojected and manual PoI
5. Create a segmentation mask from a field template and the homography
6. Create debug images (optional)
7. Save the results: *mask, theta and PoI* (and *debug images*)
8. Generate onehote masks from the rgb masks


\* *PoI - Points of Interest*  
\** *RMSE - Root Mean Square Error*

**0. Define paths and constants**

In [4]:
DEBUG_DRAW = True
FIELD_POI_PATH = './../assets/template_pitch_points.json'
FIELD_MASK_PATH = './../assets/pitch_mask_nc4_hd.png'
FIELD_SIZE = (1280, 720)
FRAME_SIZE = (1280, 720)
NUM_CLASSES = 4

# Source dirs:
FRAMES_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/sota/football/data/frames'
ANNO_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/sota/football/data/manual_anno'

# Destination dirs:
MASKS_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/sota/football/data/masks_rgb'
ONEHOT_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/sota/football/data/masks'
LABELS_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/sota/football/data/anno'
DEBUG_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/sota/football/data/debug'

**1. Generate requests (*game name, frame ID* and *PoI*) for all games in the directory**

In [5]:
requests = generate_requests(ANNO_DIR)

**2-4. Calculate homography, project the field PoI, and calculate RMSE**

In [6]:
field_poi = load_court_poi(FIELD_POI_PATH, normalize=False)
field_poi = np.expand_dims(field_poi, 0)

for game, frames in requests.items():
    print ('Calculating homography for {}...'.format(game))
    
    for frame_id, values in frames.items():
        manual_poi = values['manual_poi']
        theta = calculate_homography(field_poi[0], manual_poi)
        
        if theta is not None:
            poi = cv2.perspectiveTransform(field_poi, theta)[0]
            nonzero = find_nonzero_points(manual_poi, FOOTBALL_PITCH_IGNORE_POINTS)
            rmse = calculate_reprojection_rmse(manual_poi, poi, nonzero, FIELD_SIZE)
            values['poi'] = poi
            values['rmse'] = rmse
            values['theta'] = theta

Calculating homography for VTB_mol_Ufa_at_Dinamo...
Calculating homography for VTB_mol_Ural_at_Dinamo...
Calculating homography for VTB_mol_Sochi_at_Dinamo...
Calculating homography for Football_Forte_at_Dinamo...
Calculating homography for VTB_mol_Rostov_at_Dinamo...


**5-7. Create segmentation masks, debug images, and save them**

In [7]:
FIELD_MASK = load_court_mask(FIELD_MASK_PATH, FIELD_SIZE, inter=cv2.INTER_NEAREST)
counter = 0

for game, frames in requests.items():
    print ('Creating masks for {}...'.format(game))
    game_masks_dir = os.path.join(MASKS_DIR, game)
    game_labels_dir = os.path.join(LABELS_DIR, game)
    game_debug_dir = os.path.join(DEBUG_DIR, game)
    
    if not os.path.exists(game_masks_dir):
        os.makedirs(game_masks_dir)
    if not os.path.exists(game_labels_dir):
        os.makedirs(game_labels_dir)
    if DEBUG_DRAW and not os.path.exists(game_debug_dir):
        os.makedirs(game_debug_dir)
    
    for frame_id, values in frames.items():
        theta = values['theta']
        poi = values['poi']
        rmse = values['rmse']
        
        if theta is None:
            print ('Skipping {}/{}'.format(game, frame_id))
            continue
            
        # Rescale theta (homography) to the image size:
        scaled_theta = rescale_theta(FIELD_SIZE, FRAME_SIZE, theta)

        # Project the field image (template) to the frame coordinates:
        mask = cv2.warpPerspective(FIELD_MASK, scaled_theta, FIELD_SIZE, flags=cv2.INTER_NEAREST)
        
        # Debug drawing:
        debug_img = None
        if DEBUG_DRAW:
            img_path = os.path.join(FRAMES_DIR, game, frame_id + '.jpeg')
            img = cv2.imread(img_path, 1)
            debug_img = debug_draw(img, mask, poi, rmse)
        
        # Save the anno:
        anno_path = os.path.join(game_labels_dir, frame_id + '.json')
        label = {
            'homo': theta,
            'poi': poi,
            'rmse': rmse
        }
        with open(anno_path, 'w') as f:
            json.dump(label, f, cls=NumpyEncoder, indent=2)
            
        # Save the mask:
        mask_path = os.path.join(game_masks_dir, frame_id + '.png')
        cv2.imwrite(mask_path, mask)
        
        # Save the debug image:
        if debug_img is not None:
            debug_path = os.path.join(game_debug_dir, frame_id + '.jpeg')
            cv2.imwrite(debug_path, debug_img, [cv2.IMWRITE_JPEG_QUALITY, 90])
            
        counter += 1
            
print ('All done! Total masks:', counter)

Creating masks for VTB_mol_Ufa_at_Dinamo...
Skipping VTB_mol_Ufa_at_Dinamo/img-00013
Creating masks for VTB_mol_Ural_at_Dinamo...
Skipping VTB_mol_Ural_at_Dinamo/img-00018
Skipping VTB_mol_Ural_at_Dinamo/img-00065
Skipping VTB_mol_Ural_at_Dinamo/img-00079
Skipping VTB_mol_Ural_at_Dinamo/img-00084
Creating masks for VTB_mol_Sochi_at_Dinamo...
Skipping VTB_mol_Sochi_at_Dinamo/img-00088
Creating masks for Football_Forte_at_Dinamo...
Skipping Football_Forte_at_Dinamo/img-00045
Skipping Football_Forte_at_Dinamo/img-00049
Skipping Football_Forte_at_Dinamo/img-00050
Skipping Football_Forte_at_Dinamo/img-00052
Skipping Football_Forte_at_Dinamo/img-00065
Creating masks for VTB_mol_Rostov_at_Dinamo...
Skipping VTB_mol_Rostov_at_Dinamo/img-00004
Skipping VTB_mol_Rostov_at_Dinamo/img-00006
Skipping VTB_mol_Rostov_at_Dinamo/img-00008
Skipping VTB_mol_Rostov_at_Dinamo/img-00022
Skipping VTB_mol_Rostov_at_Dinamo/img-00068
Skipping VTB_mol_Rostov_at_Dinamo/img-00071
Skipping VTB_mol_Rostov_at_Dinamo/i

**8. Generate onehote masks from the rgb masks**

In [18]:
!rm -rf $ONEHOT_DIR
!cp -rf $MASKS_DIR $ONEHOT_DIR
generate_onehot(ONEHOT_DIR, NUM_CLASSES)

Done! Processed masks: 268
